In [1]:
def sql(query):
    !sqlite3 function_survey/output/function_survey.db "{query}"

In [2]:
sql(".schema")

CREATE VIEW used_func_names(file, outer_function, name) AS
SELECT 
  cregit_names.file AS file, cregit_names.function_name AS outer_function,
  cregit_names.token_name AS name
FROM cregit_names INNER JOIN cregit_functions ON cregit_names.token_name = cregit_functions.name;
CREATE VIEW test_funcs AS SELECT * from cregit_functions WHERE name LIKE '%test%'
/* test_funcs(file,name) */;
CREATE VIEW tested_funcs AS SELECT * FROM test_funcs JOIN cregit_calls ON cregit_calls.caller = test_funcs.name
/* tested_funcs(file,name,"file:1",caller,callee) */;
CREATE TABLE cregit_functions (file,name);
CREATE INDEX idx_functions ON cregit_functions (file, name);
CREATE TABLE cregit_calls (file,caller,callee);
CREATE INDEX idx_calls ON cregit_calls (file,caller,callee);
CREATE TABLE cregit_includes (file,include);
CREATE TABLE cregit_specifiers (file,name,specifier);
CREATE TABLE cregit_identifiers (file,function,identifier);
CREATE INDEX idx_identifiers ON cregit_identifiers (file,function,identifier)

In [3]:
#sql("""DROP VIEW functions;""")

In [4]:
sql("""
CREATE VIEW IF NOT EXISTS functions AS
    WITH definitions AS (
        SELECT *, COUNT(*) AS times_defined FROM cregit_functions
        GROUP BY file, name
    ), specifiers AS (
        SELECT file,name, COUNT(*) AS specifier_count FROM cregit_specifiers
        WHERE specifier = 'static'
        GROUP BY file, name
    ), pairs AS (SELECT definitions.file, definitions.name, times_defined,
        IFNULL(specifier_count, 0) AS specifier_count 
        FROM definitions LEFT JOIN specifiers
        ON definitions.file = specifiers.file AND definitions.name = specifiers.name
    )
SELECT file, name, times_defined, specifier_count,
    CASE
        WHEN times_defined = specifier_count THEN 'static'
        WHEN specifier_count = 0 THEN 'non-static'
        ELSE 'varies' END
    AS is_static
FROM pairs
;""")

In [5]:
# Finds the problematic functions that are marked as both static and not static
sql("""SELECT * FROM functions WHERE is_static = 'varies';""")

arch/arm/mach-omap2/board-n8x0.c|n8x0_mmc_init|2|1|varies
arch/csky/kernel/stacktrace.c|walk_stackframe|2|1|varies
arch/h8300/kernel/irq.c|setup_vector|2|1|varies
arch/ia64/kernel/topology.c|arch_register_cpu|2|1|varies
arch/mips/kvm/mmu.c||2|1|varies
arch/powerpc/kernel/fadump.c|fadump_cma_init|2|1|varies
arch/powerpc/mm/book3s64/hash_utils.c|get_paca_psize|2|1|varies
arch/powerpc/platforms/cell/axon_msi.c|axon_msi_debug_setup|2|1|varies
arch/x86/kernel/topology.c|arch_register_cpu|2|1|varies
arch/x86/pci/xen.c|xen_setup_pci_msi|2|1|varies
block/bfq-cgroup.c|bfqg_and_blkg_get|2|1|varies
drivers/acpi/acpica/evxface.c||12|1|varies
drivers/acpi/acpica/nsxfeval.c||7|2|varies
drivers/acpi/acpica/nsxfname.c||3|1|varies
drivers/acpi/acpica/rsxface.c||8|1|varies
drivers/bus/mips_cdmm.c||2|1|varies
drivers/gpu/drm/drm_mm.c||3|2|varies
drivers/infiniband/hw/qib/qib_debugfs.c||3|2|varies
drivers/mmc/host/sdhci-pltfm.c|sdhci_get_compatibility|2|1|varies
drivers/pci/pci-sysfs.c|pci_create_resource

In [13]:
sql("""SELECT name, COUNT (DISTINCT file) AS num_files, COUNT (DISTINCT is_static) FROM functions
GROUP BY name
HAVING num_files > 1 AND (is_static = 'varies' or COUNT(DISTINCT is_static) > 1)
ORDER BY num_files DESC
LIMIT 30;""")

|170|3
usage|84|2
f|71|2
platform_init|41|2
prom_init|33|2
paging_init|32|2
run_test|30|2
die|29|2
pcibios_map_irq|28|2
machine_restart|27|2
vidioc_s_input|26|2
vidioc_g_input|26|2
machine_power_off|26|2
machine_halt|26|2
vidioc_g_fmt_vid_cap|25|2
get_sigframe|25|2
vidioc_try_fmt_vid_cap|24|2
vidioc_s_fmt_vid_cap|24|2
vidioc_enum_input|24|2
handle_signal|24|2
do_signal|23|2
test|21|2
vidioc_g_std|20|2
parse_opts|20|2
sig_handler|19|2
show_cpuinfo|19|2
smp_prepare_cpus|18|2
parse_options|18|2
do_page_fault|18|2
timer_interrupt|17|2
